# 1.　ngrokのauth_tokeを設定して前回の内容を実行[事前準備、設定]

In [1]:
#@title #1.　ngrokのauth_tokeを設定して前回の内容を実行[事前準備、設定]

!pip install --upgrade pip #12/27 21:46追加

# 1,2 Flask, pyngrokのインストール
!pip install flask pyngrok
#  必要なモジュールのimport
import os
# flaskからFlaskクラスをインポート
from flask import Flask
# pyngrokからngrokモジュールとconfモジュールをインポート
from pyngrok import ngrok, conf
# 設定
# ngrokトークンを設定
conf.get_default().auth_token = "" #ここに自分のNgrokトークンを入れてください！！！！！


  Using cached MarkupSafe-3.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.0 kB)
Using cached MarkupSafe-3.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (20 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.0
    Not uninstalling markupsafe at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.


## 1-2 Flask-SQLAlchemyのインストール

In [2]:
# @title 1-2 Flask-SQLAlchemyのインストール
!pip install --upgrade flask-sqlalchemy

  Using cached MarkupSafe-3.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.0 kB)
Using cached MarkupSafe-3.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (20 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.0
    Not uninstalling markupsafe at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.


## 1-3 OpenAIライブラリのインストール

In [3]:
# @title #1-3 OpenAIライブラリのインストール
# https://platform.openai.com/docs/api-reference/introduction
!pip install openai==1.55.3 httpx==0.27.2 anyio==3.7.0

  Using cached anyio-3.7.0-py3-none-any.whl.metadata (4.7 kB)
Using cached anyio-3.7.0-py3-none-any.whl (80 kB)
  Attempting uninstall: anyio
    Found existing installation: anyio 3.6.2
    Not uninstalling anyio at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'anyio'. No files were found to uninstall.


## 4.アプリケーションを初期化する

In [4]:
#@title　#4.アプリケーションを初期化する

from flask import g, render_template


# テンプレートと静的ファイルの置き場所を定数に代入
TEMPLATE_PATH = 'flask_blog/templates'
STATIC_PATH = 'flask_blog/static'
# Flaskのインスタンスを作るときにそれを利用してファイルパスを設定する
app = Flask(__name__, template_folder=TEMPLATE_PATH, static_folder=STATIC_PATH)

## 5.Flask-SQLAlcyemyをアプリに設定をする

In [5]:
#@title　#5.Flask-SQLAlcyemyをアプリに設定をする

from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
  pass #これは何にもしないと言う意味

# SQLAlchemyをインスタンス化
db = SQLAlchemy(model_class=Base)

# SQLiteデータベースをappに設定する
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///sample_db.sqlite"
#  appを拡張とともに初期化
db.init_app(app)

## 6-1. Tagモデル, Blog_Tagテーブル（中間テーブル）を作る

In [6]:
#@title　#7. Tagモデル, Blog_Tagテーブル（中間テーブル）を作る
from __future__ import annotations
from sqlalchemy import Table
from sqlalchemy import ForeignKey
from sqlalchemy import Column
from sqlalchemy.orm import relationship
from sqlalchemy import  DateTime, Integer, String
from sqlalchemy.orm import Mapped, mapped_column, validates

association_table = Table(
    "blog_tag",
    Base.metadata,
    Column("blog_id", ForeignKey("blog.id")),
    Column("tag_id", ForeignKey("tag.id")),
)

class Tag(db.Model):
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(nullable=False, unique=True)
    created_at: Mapped[int] = db.Column(db.DateTime(timezone=True))

    #ここにブログの関連　ああーーーーなるほど一旦整理してずにしてきます！ありがとうございます！！！これでこのあと頑張ってみます！はい！おつかれさmでしたー
    blogs: Mapped[list[Blog]] = relationship(secondary=association_table)  #tag -> Blog

    @validates("name")
    def validate_existance(self, key, value):
        if not value:
            raise ValueError(key)

        return value
    def to_json(self):
        return {
            "id": self.id,
            "name": self.name
        }



## 6-2. Blogモデルを作る

In [7]:
#@title　#6. Blogモデルを作る

class Blog(db.Model):
    id: Mapped[int] = mapped_column(primary_key=True)
    title: Mapped[str]  = mapped_column(nullable=False)
    body: Mapped[str] = mapped_column(nullable=False)
    user_name: Mapped[str] = mapped_column(nullable=False)
    created_at: Mapped[int] = db.Column(db.DateTime(timezone=True))
    comments = db.relationship('Comment', back_populates='blog') # blog.commentsで関係するコメントを取れる
    tags: Mapped[list[Tag]] = relationship(secondary=association_table) #blog.tags

    # https://docs.sqlalchemy.org/en/20/orm/mapped_attributes.html#simple-validators
    @validates("title", "body", "user_name")
    def validate_existance(self, key, value):
        if not value:
            raise ValueError(key)

        return value

## 6-3.  Commentモデルを作る

In [8]:
#@title　#6. Commentモデルを作る
class Comment(db.Model):
    id: Mapped[int] = mapped_column(primary_key=True)
    blog_id: Mapped[str]  = mapped_column(db.ForeignKey(Blog.id))
    blog = db.relationship('Blog', back_populates='comments') # comment.blogで対象blogを取れる
    body: Mapped[str] = mapped_column(nullable=False)
    user_name: Mapped[str] = mapped_column(nullable=False)
    created_at: Mapped[int] = db.Column(db.DateTime(timezone=True))

    @validates("blog_id", "body", "user_name")
    def validate_existance(self, key, value):
        if not value:
            raise ValueError(key)

        return value

## 7. モデルをもとにテーブルを作成する

In [9]:
#@title #7.モデルをもとにテーブルを作成する

# アプリケーションが実行されるコンテキスト内で行うという指定のwith
with app.app_context():
  # db.drop_all() # 今まで作成したテーブルを一度すべて消す。6でBlogモデルに変更を加えてない場合は不要
  #  テーブルを作成
  db.create_all()

## 8. タグテーブルにデータを入れる

In [10]:
#@title #8. タグテーブルにデータを入れる

from sqlalchemy import delete
with app.app_context():
    db.session.execute(db.delete(Tag)) # 同じデータが作られるのを防ぐために一度全部消す
    for name in ["日記", "ガラスペン", "文房具"]:#なるほど
        tag = Tag(name=name)
        db.session.add(tag)
        db.session.commit()

/tmp/ipykernel_1044/2919490625.py:7: SAWarning: relationship 'Blog.tags' will copy column blog.id to column blog_tag.blog_id, which conflicts with relationship(s): 'Tag.blogs' (copies blog.id to blog_tag.blog_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="blogs"' to the 'Blog.tags' relationship. (Background on this warning at: https://sqlalche.me/e/20/qzyx) (This warning originated from the `configure_mappers()` process, which was invoked automatically in response to a user-initiated operation.)
  tag = Tag(name=name)
/tmp/ipykernel_1044/2919490625.py:7: SAWarning: relationship 'Blog.tags' will copy column tag.id to column bl

## 9. アプリケーションでデータを読み出す+テンプレートにデータを渡すプログラミング

### 9-1. ルート/, GET /blogs 一覧画面（フォーム付き）, POST /blogs データ作成機能

In [11]:
#@title #9-1. ルート/, GET /blogs 一覧画面（フォーム付き）, POST /blogs データ作成機能 （前回と同じ）


from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from flask import g, render_template, redirect, url_for, request, flash, session

#セッションを設定する
app.secret_key = 'timestamp'
app.permanent_session_lifetime = timedelta(days=5)
# /（ルート）にアクセスしたら次に書く関数(home)を実行するという宣言
@app.route("/")
def home():
    return render_template('home.html')

# /blogs にHTTPメソッドがGETでアクセスしたblog_index関数を実行する
@app.get("/posts")
def blog_index():
   #データベースからデータを引き出す（オブジェクトのリストの形で帰ってくる）
    # https://msiz07-flask-docs-ja.readthedocs.io/ja/latest/patterns/sqlalchemy.html#declarative
    blogs = Blog.query.all()
    tags = Tag.query.all()
    # テンプレートにblogs変数を渡す
    return render_template('posts.html', blogs = blogs,tags =tags, blog = Blog())

# /blogs にHTTPメソッドがPOSTでアクセスしたblog_create関数を実行する
@app.post("/posts")
def blog_create():

    # フォームの値を取得してバリデーションを行う
    title = request.form["title"]
    user_name = request.form["user_name"]
    body = request.form["body"]
    blog = Blog()
    try:
        blog = Blog(title = title, user_name = user_name, body = body)
        db.session.add(blog)
        db.session.commit()
    except ValueError as ve:
      flash(f"入力が不足しています#{ve}")
    except Exception as e:
       flash(f"予期せぬエラー #{e}")
    blogs =  Blog.query.all()
    # blogs.htmlテンプレートを描画
    return render_template('posts.html', blogs = blogs, blog = blog)


### 9-2 GET /blogs/:id 更新用フォーム、Post(気持ちはpatch) /blogs/:id 更新処理

In [12]:
# @title 9-2 GET /blogs/:id 更新用フォーム、Post(気持ちはpatch) /blogs/:id 更新処理

@app.get('/posts/<string:id>')
def get_blog(id):
    # 指定のidのblogオブジェクトを取得する
    blog = db.get_or_404(Blog, id)
    # https://flask-sqlalchemy.readthedocs.io/en/3.1.x/queries/#queries-for-views
    return render_template('post.html', blog = blog)

@app.post('/posts/<string:id>')
def update_blog(id):
    # 指定のidのblogオブジェクトを取得する
    blog = db.get_or_404(Blog, id)

    # 更新処理の場合
    if request.form.get('_method') == 'PATCH':
        try:
            blog.title = request.form["title"]
            blog.user_name = request.form["user_name"]
            blog.body = request.form["body"]
            db.session.commit()
            flash("データを更新しました")
        except ValueError as ve:
            flash(f"入力が不足しています#{ve}")
        except Exception as e:
            flash(f"予期せぬエラー #{e}")
        # 編集画面にGETでリダイレクトする
        return redirect(url_for("get_blog", id=id))
    # 削除リクエストの場合
    elif request.form.get('_method') == 'DELETE':
        db.session.delete(blog)
        db.session.commit()
        # 一覧画面にGETでリダイレクトする
        return redirect(url_for("blog_index"))

### 9-3. GET /comments 一覧画面（フォーム付き）, POST /comments データ作成機能

In [13]:
#@title #9-3. GET /comments 一覧画面（フォーム付き）, POST /comments データ作成機能


# /comments にHTTPメソッドがGETでアクセスしたcomment_index関数を実行する
@app.get("/posts/<string:blog_id>/comments")
def comment_index(blog_id):
    #データベースからデータを引き出す（オブジェクトのリストの形で帰ってくる）
    # https://msiz07-flask-docs-ja.readthedocs.io/ja/latest/patterns/sqlalchemy.html#declarative
    # URLで指定されたblog_idのblogオブジェクトを取得する
    blog = db.get_or_404(Blog, blog_id)
    # テンプレートにcomments変数を渡す
    return render_template('comments.html', comments = blog.comments, comment = Comment()) #コメントは

# /comments にHTTPメソッドがPOSTでアクセスしたcomments_create関数を実行する
@app.post("/posts/<string:blog_id>/comments")
def comment_create(blog_id):
    # URLで指定されたblog_idのblogオブジェクトを取得する
    blog = db.get_or_404(Blog, blog_id)
    # フォームの値を取得してCommentモデルのオブジェクトをつくる
    user_name = request.form["user_name"]
    body = request.form["body"]
    try:
        comment = Comment(blog_id = blog.id, user_name = user_name, body = body)
        db.session.add(comment)
        db.session.commit()
    except ValueError as ve:
      flash(f"入力が不足しています#{ve}")
    except Exception as e:
       flash(f"予期せぬエラー #{e}")
    # blog詳細画面にリダイレクト
    return redirect(url_for("get_blog", id = blog.id))

### 9-4 GET /comments/:id 更新用フォーム、Post(気持ちはpatch) /comments/:id 更新処理

In [14]:
# @title #9-4 GET /comments/:id 更新用フォーム、Post(気持ちはpatch) /comments/:id 更新処理

@app.get('/posts/<string:blog_id>/comments/<string:id>')
def get_comment(blog_id, id):
    # 指定のidのcommentオブジェクトを取得する　comment変数に代入
    comment = db.get_or_404(Comment, id)
    # https://flask-sqlalchemy.readthedocs.io/en/3.1.x/queries/#queries-for-views
    # テンプレートにcomment変数を渡す
    return render_template('comment.html', comment = comment)

@app.post('/posts/<string:blog_id>/comments/<string:id>')
def update_comment(blog_id, id):
    # 指定のidのcommentオブジェクトを取得する　comment変数に代入
    comment = db.get_or_404(Comment, id)

    # 更新処理の場合
    if request.form.get('_method') == 'PATCH':
        try:
             comment.user_name = request.form["user_name"]
             comment.body = request.form["body"]
             db.session.commit()
             flash("データを更新しました")
        except ValueError as ve:
             flash(f"入力が不足しています#{ve}")
        except Exception as e:
             flash(f"予期せぬエラー #{e}")
        # 編集画面にGETでリダイレクトする
        return redirect(url_for("get_comment", blog_id=blog_id, id=id))
    # 削除リクエストの場合
    elif request.form.get('_method') == 'DELETE':
        db.session.delete(comment)
        db.session.commit()
        # 一覧画面にGETでリダイレクトする
        return redirect(url_for("get_blog", id = blog_id))

https://flask.palletsprojects.com/en/stable/patterns/javascript/

### 9-4 Post /blogs/:id/tag フォームで送られてきたタグをブログに紐づける

In [15]:
# @title #9-4 Post /blogs/:id/tag フォームで送られてきたタグをブログに紐づける
@app.post('/posts/<string:id>/tag')
def create_tag(id):

  blog = db.get_or_404(Blog, id)
  tag_name = request.form["name"]
  tag = Tag.query.filter_by(name=tag_name).first() # nameで登録済みのタグを探す
  if tag is None:
    tag = Tag(name = tag_name) # もしなければTagを作る
    db.session.add(tag)
    db.session.commit()
  blog.tags.append(tag)  # blogにタグを紐づける
  db.session.commit()
  # 確認コード例 (SQLAlchemyの場合)
  print(blog.tags)  # データベースから取得した後、正しいタグリストが含まれているか

  return redirect(url_for("get_blog", id = blog.id))

## 10.アプリケーションを立ち上げる

In [19]:
#@title #10.アプリケーションを立ち上げる

if __name__ == "__main__":
    public_url = ngrok.connect(8000)
    print(f"ngrok URL: {public_url}")
    app.run(port=8000)

ngrok URL: NgrokTunnel: "https://046e-3-220-159-34.ngrok-free.app" -> "http://localhost:8000"
 * Serving Flask app '__main__'
 * Debug mode: off
 * Running on http://127.0.0.1:8000
Press CTRL+C to quit


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5456fa35-ea0f-4cee-9aed-01d78c91284d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>